In [1]:
#输入n：物品件数；c:最大承重为c的背包；w:各个物品的重量；v:各个物品的价值
#返回（c+1）*（n+1）的matrix
def knapsack(c,n,v,w):
    matrix = [[0 for i in range(c+1)] for i in range(n+1)]
    for i in range(1,n+1):
        for j in range(1,c+1):
            matrix[i][j] = matrix[i-1][j]
            if (j -w[i-1]>=0) and matrix[i-1][int(j-w[i-1])]+ v[i-1] > matrix[i][j]:
                matrix[i][j] = matrix[i-1][int(j-w[i-1])]+ v[i-1]
    return matrix

In [2]:
#输入某一instance下的matrix，得到该instance下每个物品的True 和False的list
def tflist(matrix,c,w):
    list = []
    j =c
    list = [0 for i in range(50)]
    for i in range(1,51):
        if matrix[int(51-i)][int(j)]> matrix[int(50-i)][int(j)]:
            list[50-i] = 1
            if j>= w[50-i]:
                j -= w[50-i]
    return list

In [224]:
#在假设全部bag容量均为c的情况下，Combine each instances and the true-false list of each instances to get new CSV.
import csv
parsed_lines = [line.strip('\n').split(',') for line in open('Knapsack_Instances.csv','r')]
def get_csv(c):
    totalline = []
    for i in range(1,501):
        w =[float(a) for a in parsed_lines[3*i-3]]
        v = [float(b) for b in parsed_lines[3*i-2]]
        matrix = knapsack(c, 50, v,w)
        decision= tflist(matrix,c,w)
        totalline.append(w)
        totalline.append(v)
        totalline.append(decision)
    with open("output%d.csv"%(c), "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(totalline)


In [226]:
#假设bag 容量分别为U[100,150]，生成50个csv
for c in range(100,151):
    get_csv(c)

In [99]:
#用蒙特卡洛生成一组500个随机数，求得500instances总最佳rewards
#设包含一组随机数的array为d
import numpy as np
def get_totalbestrewards_montecarlo(d):
    totalbestrewards=0
    for i in range(1,501):
        w =[float(a) for a in parsed_lines[3*i-3]]
        v = [float(b) for b in parsed_lines[3*i-2]]
        matrix = knapsack(d[i-1], 50, v,w)
        totalbestrewards+= matrix[-1][-1]
    return totalbestrewards


118675.03999999996

In [229]:
#重复上一步，用蒙特卡洛生成50组随机数，分别求得每组随机数下500个instances总最佳rewards
#设包含50组随机数的list为D
#设包含50组随机数下总最佳rewards的list为D_totalbestrewards
D = [np.random.randint(100,151,500) for i in range(50)]
D_totalbestrewards = []
for d in D:
    D_totalbestrewards.append(get_totalbestrewards_montecarlo(d))
D_totalbestrewards

[119115.75000000001,
 116392.06999999998,
 119093.17,
 117082.28999999992,
 118739.51000000005,
 117901.31999999998,
 117265.55999999998,
 117297.88999999998,
 118501.04,
 117387.10999999991,
 117473.55000000006,
 116239.06999999998,
 116927.61999999988,
 118745.46000000004,
 117319.78000000004,
 116342.54999999987,
 116850.3400000001,
 117615.15999999995,
 116947.75999999991,
 116512.23999999996,
 117634.93000000004,
 116422.13,
 117756.23999999998,
 116800.2,
 116989.83999999988,
 117299.16999999988,
 117290.96999999999,
 116163.76,
 116997.82999999984,
 117922.52999999998,
 117892.62999999998,
 116793.83999999995,
 117681.08,
 116939.91999999993,
 117197.09000000004,
 116312.67,
 118185.04000000001,
 116586.33000000002,
 116591.71000000018,
 117694.22000000003,
 117677.84000000011,
 117735.53999999998,
 116964.7499999999,
 118410.54,
 117562.04999999996,
 117812.96999999996,
 117339.29999999984,
 116877.20999999993,
 117366.14000000003,
 118078.64000000004]

In [227]:
#对于某一个c值下500个instances的decision(csv file),分别用50组随机数做为真实背包容量，计算我们在该c值下的50个总rewards
def get_decisionrewards(c):
    parsed_lines = [line.strip('\n').split(',') for line in open("output%d.csv"%(c), "r")]
    D_totalrewards = []
    for j in range(50):
        d = D[j]
        total_rewards = 0
        for i in range(1,501):
            w =[float(a) for a in parsed_lines[3*i-3]]
            v =[float(b) for b in parsed_lines[3*i-2]]
            decision = [float(c) for c in parsed_lines[3*i-1]]
            decision_weight = 0
            decision_value = 0
            for p in range(50):
                decision_weight+=decision[p]*w[p]
                decision_value +=decision[p]*v[p]
            if decision_weight<=d[i-1]:
                total_rewards += decision_value 
        D_totalrewards.append(total_rewards)
    return(D_totalrewards)

In [241]:
#计算出某一个c值下，50个总rewards 的平均值，除以50个总最佳rewards的平均值，得到该c值的成绩
def get_grade(c):
    denom = sum([a for a in get_decisionrewards(c)])
    nom = sum(D_totalbestrewards)
    grade = denom/nom
    return grade

In [248]:
#计算每一个c值的成绩，选出最佳c值
def get_best_c():
    best_c = 0
    best_grade = 0
    for i in range(100,151):
        if get_grade(i)>best_grade:
            best_grade = get_grade(i)
            best_c = i
    return best_c

In [249]:
get_best_c()

101

In [2]:
import numpy as np
import csv

In [4]:
#生成500个csv文件，每个csv中包含每个instance在c=U[100,150]下的最佳decision
parsed_lines = [line.strip('\n').split(',') for line in open('Knapsack_Instances.csv','r')]
for i in range(1,501):
        w =[float(a) for a in parsed_lines[3*i-3]]
        v = [float(b) for b in parsed_lines[3*i-2]]
        totalline = []
        for c in range(100,151):
            matrix = knapsack(c, 50, v,w)
            decision= tflist(matrix,c,w)
            totalline.append(w)
            totalline.append(v)
            totalline.append(decision)
        with open("instance%d.csv"%(i), "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerows(totalline)

KeyboardInterrupt: 

In [5]:
#对每个instance的每个bag size，用100个random真实bag size算出每个bag size的total rewards，返回到一个list里
def get_rewards_instance(d,random):
    parsed_lines = [line.strip('\n').split(',') for line in open("instance%d.csv"%(d), "r")]
    D_totalrewards_ins = []
    for i in range(1,52):
        w =[float(a) for a in parsed_lines[3*i-3]]
        v =[float(b) for b in parsed_lines[3*i-2]]
        decision = [float(c) for c in parsed_lines[3*i-1]]
        decision_weight = 0
        decision_value = 0
        total_rewards = 0
        for p in range(50):
            decision_weight+=decision[p]*w[p]
            decision_value +=decision[p]*v[p]
        for j in range(len(random)):
            b = random[j]
            if decision_weight<=b:
                total_rewards += decision_value 
        D_totalrewards_ins.append(total_rewards)
    return(D_totalrewards_ins)

In [6]:
#找出list中最大的rewards，确认该rewards所用的bagsize
def get_ins_bestbagsize(rewardslist):
    bestrewards = max(rewardslist)
    bagsize = rewardslist.index(bestrewards)+100
    return bagsize

In [9]:
test_number = 50000
random = np.random.randint(100,151,test_number)

In [7]:
#Combine previous functions，对500个instance的中的每一个，找出相应最佳bagsize，生成包含500个best bagsize的list
def get_bestsize_list():
    total_bagsize_list = []
    for d in range(1,501):
        rewardslist = get_rewards_instance(d,random)
        ins_best_bagsize = get_ins_bestbagsize(rewardslist)
        total_bagsize_list.append(ins_best_bagsize)
    return(total_bagsize_list)

In [10]:
bestsize_list = get_bestsize_list()


In [11]:
#最后用新的最佳bag size list生成新的csv
def get_level4_csv(bestsize_list):
    parsed_lines = [line.strip('\n').split(',') for line in open('Knapsack_Instances.csv','r')]
    totalline = []
    for i in range(1,501):
        w =[float(a) for a in parsed_lines[3*i-3]]
        v = [float(b) for b in parsed_lines[3*i-2]]
        matrix = knapsack(bestsize_list[i-1], 50, v,w)
        decision= tflist(matrix,bestsize_list[i-1],w)
        totalline.append(w)
        totalline.append(v)
        totalline.append(decision)
    with open("level4_output50000.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(totalline)

In [12]:
get_level4_csv(bestsize_list)